<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Госпрограммы" data-toc-modified-id="Госпрограммы-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Госпрограммы</a></span></li><li><span><a href="#Планы-реализации-госпрограмм" data-toc-modified-id="Планы-реализации-госпрограмм-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Планы реализации госпрограмм</a></span></li><li><span><a href="#Порядки-предоставления-субсидий" data-toc-modified-id="Порядки-предоставления-субсидий-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Порядки предоставления субсидий</a></span></li></ul></div>

In [1]:
import os
import shutil

import time
import datetime
from datetime import date,datetime

import numpy as np
import pandas as pd
from IPython.display import display

from sqlalchemy import create_engine

In [2]:
POSTGRES_ADDRESS = '10.248.23.152'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'isu'
POSTGRES_PASSWORD = 'isupass'
POSTGRES_DBNAME = 'isu_db'

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                                                                                        password=POSTGRES_PASSWORD,
                                                                                        ipaddress=POSTGRES_ADDRESS,
                                                                                        port=POSTGRES_PORT,
                                                                                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [3]:
task_list = ['Анализ госпрограмм', 'Анализ планов реализации', 'Анализ порядков предоставления субсидий']

employees_dict = {'Отдел аграрно-продовольственной политики, природопользования и финансового сектора':\
                  ['Шибина Л.В.', 'Басинских М.В.', 'Татьянина М.И.', 'Брик И.А.', 'Царик А.М.', 'Агеева Д.А.'],
                  'Отдел развития реального сектора экономики': ['Пушилин В.А.', 'Макарова Ю.О.', 'Зубова Е.И.', 
                                                                 'Преснякова Д.С.'],
                  'Отдел развития социальной сферы': ['Евсина М.Н.', 'Колошин Д.И.', 'Петрова А.Б.', 'Слюняева Т.Б.', 
                                                      'Шавина А.В.', 'Вязникова А.В.', 'Болдырева Е.Ю.']}

In [4]:
def get_key(d, value):
    """
    Функция для восстановления ключа по значению
    :param d: словарь dict
    :param value: значение, для которого ищем ключ
    :return: ключ из словаря
    """
    for keys, values in d.items():
        if value in values:
            return keys
    else:
        return 'ФИО исполнителя нет в списке сотрудников'

In [5]:
'''Получение перечня файлов'''
def GetFileList(directory):
   
    #Получаем список файлов в переменную files
    files = os.listdir(directory)
    files
    #Получаем список файлов .xlsx
    reports = []
    for file in files:
        if file.endswith((".xlsx",".xlsm")):
            reports.append(file)
    return reports

In [6]:
def get_info_from_dir(directory, template_ind, drop_list, change_dict):
    template = pd.read_excel(f'{templates_dir}{templates[template_ind]}', sheet_name='ЧЕК-ЛИСТ2', header=4)
    main_col = 'Наименование требования'
    template = template[[main_col]]
    
    print(f'Количество чек-листов в папке: {len(GetFileList(directory))}\n')
    result = template.copy()

    for file_ind, file in enumerate(GetFileList(directory)[:]):
        try:
            temp = pd.read_excel(f'{directory}{file}', sheet_name='ЧЕК-ЛИСТ2', header=4)
            temp.rename(columns={'Статус': file}, inplace=True)
            main_col = 'Наименование требования'
            temp = temp[[main_col, file]]

            temp = temp[~temp[main_col].isin(drop_list)]        
            for key in change_dict:
                temp.loc[temp[temp[main_col].isin([key])].index, main_col] = change_dict[key]


            temp_col = temp[main_col].unique()
            result_col = result[main_col].unique()
            check = list(set(temp_col) - set(result_col))
            check_ = list(set(result_col) - set(temp_col))
#             if (check!=[]) or (check_!=[]):
#                 print(f'{file}\n')
#                 if check!=[]:
#                     print(f'Необходимо удалить/переименовать: {check}\n')
#                 if check_!=[]:
#                     print(f'Необходимо добавить: {check_}\n')            
#             else:
#                 result = pd.merge(result, temp, how='outer', on=main_col)                
            if (check!=[]):
                print(f'{file}\n')
                print(f'Необходимо удалить/переименовать: {check}\n')
            else:
                result = pd.merge(result, temp, how='outer', on=main_col)

        except PermissionError:
            print(f'\033[93m{file} редактируется другим пользователем\033[0m\n')
            
    result = result.T
    result.columns = result.iloc[0]
    result['file'] = result.index
    result.index = range(len(result.index))
    result.drop([0], inplace=True)
    result.index = range(len(result.index))
    if len(result) == len(GetFileList(directory)):
        print('Успешное завершение!')
        return result
    else:
        print('Загружены не все данные! Есть проблемы с названиями столбцов!')
        return None        

In [7]:
"""Шаблоны"""
templates_dir = r'J:\~Шаблоны/'
templates = GetFileList(templates_dir)
templates

['Шаблон заключения на госпрограмму.xlsx',
 'Шаблон заключения на план реализации.xlsx',
 'Шаблон заключения на порядок предоставления субсидий.xlsx']

In [8]:
"""Сохранение результатов"""
# Folder = r'U:\~ 09_Машинное обучение_Прогноз показателей СЭР\ЧЕК-ЛИСТЫ и DATA-SHOP\ВЫГРУЗКА ЧЕК-ЛИСТОВ/'
Folder = r'U:\ЧЕК-ЛИСТЫ и DATA-SHOP\ВЫГРУЗКА ЧЕК-ЛИСТОВ/'
File_name = 'RESULTS - Чек-листы.xlsx'

# Госпрограммы

In [16]:
template_ind = 0
directory = r'J:\~Шаблоны\Чек-листы\Госпрограммы/'

drop_list = []
change_dict = {
    '1.5. Ожидаемые результаты реализации госпрограммы/подпрограммы установлены на основе из плановых значений индикаторов (показателей)': 
    '1.5. Ожидаемые результаты реализации госпрограммы/подпрограммы установлены на основе плановых значений индикаторов (показателей)',
    '1.6.1. соответствуют лимитам, бюджету ': '1.6.1. соответствуют бюджетной росписи',      
}

result = get_info_from_dir(directory, template_ind, drop_list, change_dict)

Количество чек-листов в папке: 83

Успешное завершение!


In [17]:
result['task'] = task_list[0]
result['Отдел УЭР'] = [get_key(employees_dict, employee) for employee in result['Исполнитель'].values]
for i in result.index:
    _ = list(map(int, result.loc[i, 'Дата заключения'].split('.')))
    result.loc[i, 'date'] = date(_[-1], _[-2], 1)        
    result.loc[i, 'date_conclusion'] = date(_[-1], _[-2], _[0])
result = result[['task', 'date_conclusion', 'date', 'Отдел УЭР', 'Исполнитель', 
         'Ответственное отраслевое управление',
         'Тип согласования', 'Статус заключения', 'Номер и дата заключения в СЭД "Дело"',         
         'Наименование государственной программы', 'Номер государственной программы', 'file', ]]
result['Ответственное отраслевое управление'] = result['Ответственное отраслевое управление'].map("Управление {}".format)

In [18]:
result.columns = ['task', 'date_conclusion', 'date', 'department_uer', 'executor',
                  'department', 'type_approval',
                  'status_conclusion', 'number_date_conclusion',
                  'name_state_program', 'number_state_program', 'file']

In [19]:
result

,task,date_conclusion,date,department_uer,executor,department,type_approval,status_conclusion,number_date_conclusion,name_state_program,number_state_program,file
0,Анализ госпрограмм,2020-08-06,2020-08-01,Отдел развития реального сектора экономики,Пушилин В.А.,Управление государственной службы и кадровой р...,Согласование изменений,Не согласовано,И38/04/-1726 от 06.08.2020,Эффективное государственное управление и разви...,18,20_07_24_Чек-лист госпрограммы эффективное гос...
1,Анализ госпрограмм,2020-08-27,2020-08-01,Отдел развития реального сектора экономики,Пушилин В.А.,Управление государственной службы и кадровой р...,Согласование изменений,Не согласовано,от 27.08.2020,Эффективное государственное управление и разви...,18,20_08_24_Чек-лист госпрограммы эффективное гос...
2,Анализ госпрограмм,2020-08-27,2020-08-01,Отдел развития реального сектора экономики,Пушилин В.А.,Управление государственной службы и кадровой р...,Согласование изменений,Согласовано,от 27.08.2020,Эффективное государственное управление и разви...,18,20_09_04_Чек-лист госпрограммы эффективное гос...
3,Анализ госпрограмм,2020-09-30,2020-09-01,Отдел развития реального сектора экономики,Зубова Е.И.,Управление жилищно-коммунального хозяйства Лип...,Согласование изменений,Согласовано с замечаниями,И28/04/-2628 от 30.09.2020,Формирование современной городской среды в Лип...,20,20_09_22_ГП 20_Зубова_вер.17.xlsx
4,Анализ госпрограмм,2020-09-30,2020-09-01,Отдел развития реального сектора экономики,Зубова Е.И.,Управление строительства и архитектуры Липецко...,Согласование изменений,Не согласовано,И28/04/-2675 от 30.09.2020,Обеспечение населения Липецкой области качеств...,08,20_09_30_ГП 08_Зубова_вер.19.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...
78,Анализ госпрограмм,2020-12-11,2020-12-01,Отдел развития социальной сферы,Колошин Д.И.,Управление внутренней политики Липецкой области,Согласование изменений,Согласовано,РК № И38/05/-3367 от 11.12.2020,Реализация внутренней политики Липецкой области,10,ГП10_Колошин Д.И. 11.12.2020.xlsx
79,Анализ госпрограмм,2020-11-23,2020-11-01,Отдел развития социальной сферы,Колошин Д.И.,Управление внутренней политики Липецкой области,Согласование изменений,Согласовано,РК № И38/05/-3149 от 23.11.2020,Реализация внутренней политики Липецкой области,10,ГП10_Колошин Д.И. 23.11.2020.xlsx
80,Анализ госпрограмм,2020-07-02,2020-07-01,"Отдел аграрно-продовольственной политики, прир...",Татьянина М.И.,Управление сельского хозяйства Липецкой области,Согласование изменений,Согласовано,38/03/-1415 от 02.07.2020,Развитие сельского хозяйства и регулирование р...,13,ГП13 Татьянина 02.07.2020.xlsx
81,Анализ госпрограмм,2020-06-25,2020-06-01,Отдел развития реального сектора экономики,Пушилин В.А.,Управление инвестиций и инноваций Липецкой обл...,Согласование изменений,Согласовано с замечаниями,456789 от 25.06.2020,Модернизация и инновационное развитие экономик...,11,Заключение 25_06_20 на госпрограмму 11 Модерни...


In [21]:
result.to_sql('check_list', cnx, if_exists='append', index=False)

In [ ]:
# with pd.ExcelWriter(Folder+File_name, engine="openpyxl", mode='a') as writer:
#         result.to_excel(writer, sheet_name = 'ГП_' + str(date.today()), header=True, index=False, encoding = '1251')

# Планы реализации госпрограмм

In [22]:
template_ind = 1
directory = r'J:\~Шаблоны\Чек-листы\План реализации/'

drop_list = []
change_dict = {
    'Внесение изменений вплан реализации не предполагает внесение изменений в постановление по утчерждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)':
    'Внесение изменений в план реализации не предполагает внесение изменений в постановление по утверждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)',
    'Внесение изменений в план реализации не предполагает внесение изменений в постановление по утчерждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)':
    'Внесение изменений в план реализации не предполагает внесение изменений в постановление по утверждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)',
    
    
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:\nзаключение государственного контракта; подписание промежуточных или итоговых актов выполненных работ': 
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:',
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события: заключение государственного контракта;подписание промежуточных или итоговых актов выполненных работ': 
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:',
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события: заключение государственного контракта; подписание промежуточных или итоговых актов выполненных работ':
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:',
    
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:формирование земельного участка;проектирование;объявление конкурсов;заключение государственного контракта;подписание промежуточных или итоговых актов выполненных работ': 
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:',
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:\nформирование земельного участка; проектирование; объявление конкурсов; заключение государственного контракта; подписание промежуточных или итоговых актов выполненных работ':
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:',
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события: формирование земельного участка;проектирование;объявление конкурсов;заключение государственного контракта;подписание промежуточных или итоговых актов выполненных работ':
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:',
    
    'Контрольное событие предусмотрено в декабре:\nпри наличии контрольных событий в предыдущих периодах;для новых мероприятий, которые были включены при уточнении бюджета в осеннюю сессию;контрольное событие подразумевает проведение события (семенара, конференции, торжественного собрания и т.п.)':
    'Контрольное событие предусмотрено в декабре:',
    'Контрольное событие предусмотрено в декабре: при наличии контрольных событий в предыдущих периодах;для новых мероприятий, которые были включены при уточнении бюджета в осеннюю сессию;контрольное событие подразумевает проведение события (семенара, конференции, торжественного собрания и т.п.)':
    'Контрольное событие предусмотрено в декабре:',        
}

result = get_info_from_dir(directory, template_ind, drop_list, change_dict)

Количество чек-листов в папке: 88

Успешное завершение!


In [23]:
result['task'] = task_list[1]
result['Отдел УЭР'] = [get_key(employees_dict, employee) for employee in result['Исполнитель'].values]
for i in result.index:
    _ = list(map(int, result.loc[i, 'Дата заключения'].split('.')))
    result.loc[i, 'date'] = date(_[-1], _[-2], 1)        
    result.loc[i, 'date_conclusion'] = date(_[-1], _[-2], _[0])
result = result[['task', 'date_conclusion', 'date', 'Отдел УЭР', 'Исполнитель', 
         'Ответственное отраслевое управление',
         'Тип согласования', 'Статус заключения', 'Номер и дата заключения в СЭД "Дело"',         
         'Наименование государственной программы', 'Номер государственной программы', 'file', ]]
result['Ответственное отраслевое управление'] = result['Ответственное отраслевое управление'].map("Управление {}".format)

In [24]:
result.columns = ['task', 'date_conclusion', 'date', 'department_uer', 'executor',
                  'department', 'type_approval',
                  'status_conclusion', 'number_date_conclusion',
                  'name_state_program', 'number_state_program', 'file']

In [25]:
result

,task,date_conclusion,date,department_uer,executor,department,type_approval,status_conclusion,number_date_conclusion,name_state_program,number_state_program,file
0,Анализ планов реализации,2020-06-05,2020-06-01,Отдел развития реального сектора экономики,Макарова Ю.О.,Управление инвестиций и инноваций Липецкой обл...,Согласование изменений,Согласовано,456789 от 05.06.2020,Обеспечение инвестиционной привлекательности Л...,15,20_07_09 Заключение на ПР ГП 15 Инвестиции _ве...
1,Анализ планов реализации,2020-08-04,2020-08-01,Отдел развития реального сектора экономики,Пушилин В.А.,Управление государственной службы и кадровой р...,Согласование изменений,Не согласовано,от 04.08.2020,Эффективное государственное управление и разви...,18,20_07_29_Чек-лист план ГП 18 эффективное госуп...
2,Анализ планов реализации,2020-08-26,2020-08-01,Отдел развития реального сектора экономики,Пушилин В.А.,Управление государственной службы и кадровой р...,Согласование изменений,Согласовано,от 26.08.2020,Эффективное государственное управление и разви...,18,20_08_25_Чек-лист план ГП 18 эффективное госуп...
3,Анализ планов реализации,2020-09-25,2020-09-01,Отдел развития реального сектора экономики,Зубова Е.И.,Управление дорог и транспорта Липецкой области,Согласование изменений,Согласовано с замечаниями,И24-3131И38/04/-2632 от 25.09.2020,Развитие транспортной системы Липецкой области,14,20_09_22_Чек-лист_ПР ГП 14_вер.20_Зубова.xlsx
4,Анализ планов реализации,2020-09-25,2020-09-01,Отдел развития реального сектора экономики,Зубова Е.И.,Управление строительства и архитектуры Липецко...,Согласование изменений,Согласовано с замечаниями,И38/04/-2636 от 25.09.2020,Обеспечение населения Липецкой области качеств...,08,20_09_23_Чек-лист_ПР ГП 08_вер.19_Зубова.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Анализ планов реализации,2021-01-25,2021-01-01,"Отдел аграрно-продовольственной политики, прир...",Басинских М.В.,Управление финансов Липецкой области,Согласование изменений,Согласовано,И38/03/-183 от 25.01.2021,Управление государственными финансами и госуда...,19,ПР к ГП 19 ДОДЕЛАТЬ Басинских 20.01.2021.xlsx
84,Анализ планов реализации,2020-08-21,2020-08-01,"Отдел аграрно-продовольственной политики, прир...",Царик А.М.,Управление финансов Липецкой области,Согласование изменений,Согласовано,И46/02-02/-2521 от 21.08.2020,Управление государственными финансами и госуда...,19,ПР К ГП 19 Царик 24.08.2020 .xlsx
85,Анализ планов реализации,2020-12-04,2020-12-01,"Отдел аграрно-продовольственной политики, прир...",Брик И.А.,Управление сельского хозяйства Липецкой области,Согласование изменений,Согласовано,И38/03/-3304 от 04.12.2020,Комплексное развитие сельских территорий Липец...,21,ПР к ГП 21 Брик 01.12.20г..xlsx
86,Анализ планов реализации,2020-08-11,2020-08-01,"Отдел аграрно-продовольственной политики, прир...",Брик И.А.,Управление сельского хозяйства Липецкой области,Согласование изменений,Согласовано,И38/03/-1789 от 11.08.2020,Комплексное развитие сельских территорий Липец...,21,ПР к ГП 21 Брик 11.08.2020.xlsx


In [26]:
result.to_sql('check_list', cnx, if_exists='append', index=False)

In [ ]:
with pd.ExcelWriter(Folder+File_name, engine="openpyxl", mode='a') as writer:
        result.to_excel(writer, sheet_name = 'ПР_' + str(date.today()), header=True, index=False, encoding = '1251')

# Порядки предоставления субсидий

In [27]:
template_ind = 2
directory = r'J:\~Шаблоны\Чек-листы\Субсидии/'

drop_list = ['Номер и дата версии в АИС Проект-СМАРТ Про', 
             'Дата в АИС Проект-СМАРТ Про',
             'Плановый год госпрограммы', 
             # Общие требования
             'Регламентирован весь период с даты объявления приема заявлений от претендентов до даты перечисления субсидии (гранта) получателю.',
             'в т.ч. срок приема документов от претендентов на получение субсидии (гранта) составляет не менее 10 рабочих дней.', 
             'Укажите количество потенциальных получателей субсидии (гранта)',
             'Установлен размер субсидии и (или) порядок расчета размера субсидии с указанием информации, обосновывающей ее размер (формулы расчета и порядок ее применения и т.п.).',
             'Установлен размер субсидии и (или) порядок расчета размера субсидии с указанием информации, обосновывающей ее размер (формулы расчета и порядок ее применения и т.п.).',
             # Гранты в форме субсидий
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)',
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)',
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)\nСубсидии производителям товаров (работ, услуг) (постановление от 06.09.2016 г. №887)\nСубсидии некоммерческим организациям (постановление от 07.05.2017 г. №541)',
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)\nСубсидии производителям товаров (работ, услуг) (постановление от 06.09.2016 г. №887)\nСубсидии некоммерческим организациям (постановление от 07.05.2017 г. №541)',
             'Определен порядок представления получателем субсидии/гранта отчетности о достижении результатов (показателей)', 
             'Определен срок представления получателем субсидии/гранта отчетности о достижении результатов (показателей), об осуществлении расходов',
             'Определена форма представления получателем субсидии/гранта отчетности о достижении результатов (показателей), об осуществлении расходов',             
             'Установлено право главного распорядителя устанавливать в соглашении сроки и формы представления получателем субсидии/гранта дополнительной отчетности',
             'Только для грантов в форме субсидий:',                                                 
            ]
change_dict = {'Наименование постановления': 'Наименование приказа',
               'Номер РКПД в СЭД Дело': 'Номер и дата РКПД в СЭД Дело',
               'Указаны результаты предоставления гранта/субсидии и при необходимости показатели, необходимые для достижения результатов предоставления гранта/субсидии.': 'Указаны результаты предоставления гранта/субсидии.',
               'Указаны результаты предоставления гранта и при необходимости показатели, необходимые для достижения результатов предоставления гранта.': 'Указаны результаты предоставления гранта/субсидии.',
               'Результаты и (или) показатели конкретны': 'Результаты конкретны',
               'Результаты и (или) показатели измеримы': 'Результаты измеримы',
               'Результаты и (или) показатели соответствуют федеральным, региональным проектам': 'Результаты и (или) показатели соответствуют федеральным, региональным проектам или государственным программам',               
              }

result = get_info_from_dir(directory, template_ind, drop_list, change_dict)

Количество чек-листов в папке: 48

Успешное завершение!


In [28]:
result['task'] = task_list[1]
result['Отдел УЭР'] = [get_key(employees_dict, employee) for employee in result['Исполнитель'].values]
for i in result.index:
    try:
        _ = list(map(int, result.loc[i, 'Дата заключения'].split('.')))
        result.loc[i, 'date'] = date(_[-1], _[-2], 1)        
        result.loc[i, 'date_conclusion'] = date(_[-1], _[-2], _[0])
    except Exception as e:
        print(result.loc[i, 'Дата заключения'], result.loc[i, 'file'], result.loc[i, 'Статус заключения'],)
        print(e)
result = result[['task', 'date_conclusion', 'date', 'Отдел УЭР', 'Исполнитель', 
         'Ответственное отраслевое управление',
         'Тип согласования', 'Статус заключения', 'Номер и дата заключения в СЭД "Дело"',         
         'Наименование государственной программы', 'Номер государственной программы', 'file', ]]
result['Ответственное отраслевое управление'] = result['Ответственное отраслевое управление'].map("Управление {}".format)

nan Чек-лист_Агеева_О предоставлении субсидий на иные цели управлением экологии.xlsx Согласовано
'float' object has no attribute 'split'
2021-04-27 00:00:00 Чек-лист_Болдырева_по Почетному гражданину.xlsx Согласовано
'datetime.datetime' object has no attribute 'split'


In [29]:
result.columns = ['task', 'date_conclusion', 'date', 'department_uer', 'executor',
                  'department', 'type_approval',
                  'status_conclusion', 'number_date_conclusion',
                  'name_state_program', 'number_state_program', 'file']

In [30]:
result

,task,date_conclusion,date,department_uer,executor,department,type_approval,status_conclusion,number_date_conclusion,name_state_program,number_state_program,file
0,Анализ планов реализации,2021-04-13,2021-04-01,Отдел развития реального сектора экономики,Макарова Ю.О.,Управление инвестиций и инноваций Липецкой обл...,Согласование изменений,Согласовано,И38/04/-973 от 13.04.2021,Модернизация и инновационное развитие экономик...,11,21_04_12Чек-лист Макарова на порядок ФРП.xlsx
1,Анализ планов реализации,2021-04-15,2021-04-01,Отдел развития реального сектора экономики,Макарова Ю.О.,Управление инвестиций и инноваций Липецкой обл...,Первоначальное утверждение,Не согласовано,И38/04/-993 от 15.04.2021,Модернизация и инновационное развитие экономик...,11,21_04_13Чек-лист Макарова на порядок на рабочи...
2,Анализ планов реализации,2021-04-21,2021-04-01,Отдел развития реального сектора экономики,Макарова Ю.О.,Управление инвестиций и инноваций Липецкой обл...,Согласование изменений,Согласовано,456789 от 21.04.2021,Модернизация и инновационное развитие экономик...,11,21_04_21Чек-лист Макарова на порядок ФРП.xlsx
3,Анализ планов реализации,2021-04-26,2021-04-01,Отдел развития реального сектора экономики,Макарова Ю.О.,Управление инвестиций и инноваций Липецкой обл...,Первоначальное утверждение,Согласовано,И38/04/-1112 от 26.04.2021,Модернизация и инновационное развитие экономик...,11,21_04_22Чек-лист Макарова на порядок на рабочи...
4,Анализ планов реализации,2020-08-10,2020-08-01,Отдел развития социальной сферы,Петрова А.Б.,Управление образования и науки Липецкой области,Согласование изменений,Согласовано,И34-2459И38/05/-1775 от 10.08.2020,Развитие образования Липецкой области,05,ГП 05 Порядок субсидий на создание доп мест от...
5,Анализ планов реализации,2021-03-05,2021-03-01,Отдел развития социальной сферы,Петрова А.Б.,Управление образования и науки Липецкой области,Первоначальное утверждение,Согласовано,И34-826И38/05/-562 от 05.03.2021,Развитие образования Липецкой области,05,ГП 05 Порядок упр образования по субсидиям на ...
6,Анализ планов реализации,2021-03-18,2021-03-01,Отдел развития социальной сферы,Петрова А.Б.,Управление образования и науки Липецкой области,Согласование изменений,Согласовано,№ И34-1044И38/05/-679 от 18.03.2021,Развитие образования Липецкой области,05,ГП 05 Порядок упр образования по субсидиям на ...
7,Анализ планов реализации,2021-03-26,2021-03-01,Отдел развития социальной сферы,Петрова А.Б.,Управление образования и науки Липецкой области,Согласование изменений,Согласовано,И34-1223И38/05/-779 от 26.03.2021,Развитие образования Липецкой области,05,ГП 05 Порядок упр образования по субсидиям на ...
8,Анализ планов реализации,2021-04-02,2021-04-01,"Отдел аграрно-продовольственной политики, прир...",Татьянина М.И.,Управление сельского хозяйства Липецкой области,Согласование изменений,Не согласовано,И38/03/-865 от 02.04.2021,Развитие сельского хозяйства и регулирование р...,13,ГП 13 Порядок предоставления субсидий в сельск...
9,Анализ планов реализации,2020-09-17,2020-09-01,Отдел развития социальной сферы,Вязникова А.В.,Управление культуры и туризма Липецкой области,Первоначальное утверждение,Согласовано,И31-1839/И38/05/-2561 от 17.09.2020,Развитие культуры и туризма в Липецкой области,06,Чек-лист СОНКО на проведение культ-мас. меро...


In [31]:
result.to_sql('check_list', cnx, if_exists='append', index=False)

In [ ]:
with pd.ExcelWriter(Folder+File_name, engine="openpyxl", mode='a') as writer:
        result.to_excel(writer, sheet_name = 'Субсидии_' + str(date.today()), header=True, index=False, encoding = '1251')